In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [8]:
!pip install transformers Pillow torch requests

In [9]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
from PIL import Image
import requests
import json
import random
import time
import os

In [10]:
model_path = "/content/drive/MyDrive/ML Final Project/Models/instructblip-flan-t5-xl"
processor_path = "/content/drive/MyDrive/ML Final Project/Processor/instructblip-flan-t5-xl"

if not os.path.exists(model_path):
    model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-flan-t5-xl")
    model.save_pretrained(model_path)

else:
    model = InstructBlipForConditionalGeneration.from_pretrained(model_path)

if not os.path.exists(processor_path):
    processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-flan-t5-xl")
    processor.save_pretrained(processor_path)
else:
    processor = InstructBlipProcessor.from_pretrained(processor_path)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

InstructBlipForConditionalGeneration(
  (vision_model): InstructBlipVisionModel(
    (embeddings): InstructBlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): InstructBlipEncoder(
      (layers): ModuleList(
        (0-38): 39 x InstructBlipEncoderLayer(
          (self_attn): InstructBlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): InstructBlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        

In [ ]:
url = "https://raw.githubusercontent.com/salesforce/LAVIS/main/docs/_static/Confusing-Pictures.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
prompt = "Generate caption"
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

outputs = model.generate(
        **inputs,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.5,
        length_penalty=1.0,
        temperature=0,
)
generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
print(generated_text)

a man is ironing clothes in the back of a yellow suv


In [13]:
# random.seed(42)

with open('/content/drive/MyDrive/ML Final Project/captions_val2014.json', 'r') as file:
    data = json.load(file)


threshold = 249
images_data = data.get('images', [])

# random.shuffle(images_data)
subset_size = len(images_data) // 5
selected_images = images_data[:subset_size]
#print(f"subset size: {subset_size}")
generated_captions = []

count = 0
for image_info in selected_images:
    start = time.time()
    image_url = image_info.get('url')

    image_id = image_info.get('id')
    resp = requests.get(image_url, stream=True)
    if not resp.ok:
        #print(f"corrupt url : {image_url}")
        continue
    #image = Image.open(io.BytesIO(requests.get(image_url, stream=True).content))
    else:
        image = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")
    #print(image_id)
    prompt = "Generate caption"
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.5,
        length_penalty=1.0,
        temperature=0,
        )
    generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

    image_id = image_info.get('id')
    generated_captions.append({"image_id": image_id, "caption": generated_text})
    count = count+1
    end = time.time()
    print(f"Processed Image: {count}/{threshold+1} \t Time Taken:{end-start}s")
    with open('/content/drive/MyDrive/ML Final Project/captions_instructblip_flan-t5_results.json', 'w') as result_file:
        json.dump(generated_captions, result_file)
    if count > threshold:
        break




Processed Image: 1/250 	 Time Taken:10.18513011932373s
Processed Image: 2/250 	 Time Taken:10.530593872070312s
Processed Image: 3/250 	 Time Taken:10.78481149673462s
Processed Image: 4/250 	 Time Taken:10.018308162689209s
Processed Image: 5/250 	 Time Taken:10.29405164718628s
Processed Image: 6/250 	 Time Taken:11.361699104309082s
Processed Image: 7/250 	 Time Taken:9.810382604598999s
Processed Image: 8/250 	 Time Taken:12.277159214019775s
Processed Image: 9/250 	 Time Taken:9.9524405002594s
Processed Image: 10/250 	 Time Taken:11.846720457077026s
Processed Image: 11/250 	 Time Taken:11.916692733764648s
Processed Image: 12/250 	 Time Taken:9.967954874038696s
Processed Image: 13/250 	 Time Taken:10.051883697509766s
Processed Image: 14/250 	 Time Taken:11.46199345588684s
Processed Image: 15/250 	 Time Taken:10.210789442062378s
Processed Image: 16/250 	 Time Taken:9.951893329620361s
Processed Image: 17/250 	 Time Taken:9.96390175819397s
Processed Image: 18/250 	 Time Taken:9.7653043270111